# 01. Parameter Extraction
## SYLVA Framework - Mediterranean Rapid Fire Spread Forecasting

![SYLVA](https://img.shields.io/badge/SYLVA-v1.0.0-blue)
![DOI](https://img.shields.io/badge/DOI-10.5281%2Fzenodo.18627186-green)

This notebook demonstrates how to extract the nine SYLVA parameters from various data sources.

In [ ]:
# Install required libraries
!pip install numpy pandas matplotlib seaborn

In [ ]:
# Import libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add project path
sys.path.insert(0, os.path.abspath('..'))

# Import SYLVA modules
from sylva_fire.parameters.fuel_moisture import FuelMoistureCalculator
from sylva_fire.parameters.fuel_structure import FuelStructureCalculator
from sylva_fire.parameters.atmospheric import AtmosphericCalculator
from sylva_fire.parameters.terrain import TerrainCalculator
from sylva_fire.parameters.drought import DroughtCodeCalculator

print("✅ Libraries imported successfully")

## 1. Live Fuel Moisture (LFM)

**Source**: Sentinel-2 NDWI

**Equation**:
$$LFM = a \times NDWI + b$$
$$NDWI = \frac{\rho_{Green} - \rho_{NIR}}{\rho_{Green} + \rho_{NIR}}$$

In [ ]:
# Initialize calculator
fm_calc = FuelMoistureCalculator()

# Simulate NDWI values
ndwi_values = np.linspace(-0.5, 0.5, 100)
lfm_values = [fm_calc.estimate_lfm_from_ndwi(ndwi) for ndwi in ndwi_values]

# Plot LFM vs NDWI
plt.figure(figsize=(10, 6))
plt.plot(ndwi_values, lfm_values, 'b-', linewidth=2)
plt.axhline(y=85, color='r', linestyle='--', label='Critical threshold (85%)')
plt.axhline(y=60, color='orange', linestyle='--', label='Extreme threshold (60%)')
plt.xlabel('NDWI')
plt.ylabel('Live Fuel Moisture (%)')
plt.title('LFM Estimation from Sentinel-2 NDWI')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

In [ ]:
# Classify LFM hazard
lfm_example = 65
hazard = fm_calc.classify_lfm_hazard(lfm_example, 'pinus_halepensis')
print(f"LFM: {lfm_example}%")
print(f"Hazard Level: {hazard['hazard_level']}")
print(f"Below Critical: {hazard['below_critical']}")

## 2. Dead Fuel Moisture (DFM)

**Source**: Weather stations + FFMC

**Equation**:
$$DFM = 147.2 - 1.27 \times FFMC$$

In [ ]:
# Calculate DFM from FFMC
ffmc_values = np.arange(70, 96, 1)
dfm_values = [fm_calc.estimate_dfm_from_ffmc(ffmc) for ffmc in ffmc_values]

plt.figure(figsize=(10, 6))
plt.plot(ffmc_values, dfm_values, 'g-', linewidth=2)
plt.axhline(y=8, color='r', linestyle='--', label='Critical threshold (8%)')
plt.axhline(y=6, color='orange', linestyle='--', label='Extreme threshold (6%)')
plt.xlabel('FFMC')
plt.ylabel('Dead Fuel Moisture (%)')
plt.title('DFM Estimation from Canadian FFMC')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## 3. Vapor Pressure Deficit (VPD)

**Source**: Temperature and Relative Humidity

**Equation**:
$$e_s = 6.1078 \times e^{\frac{17.27 \times T}{T + 237.3}}$$
$$e = e_s \times \frac{RH}{100}$$
$$VPD = e_s - e$$

In [ ]:
# Calculate VPD
temperatures = np.arange(20, 45, 2)
humidities = [20, 30, 40, 50]

plt.figure(figsize=(12, 6))

for rh in humidities:
    vpd_values = [fm_calc.calculate_vpd(t, rh) for t in temperatures]
    plt.plot(temperatures, vpd_values, 'o-', label=f'RH = {rh}%')

plt.axhline(y=25, color='r', linestyle='--', label='Critical threshold (25 hPa)')
plt.xlabel('Temperature (°C)')
plt.ylabel('VPD (hPa)')
plt.title('Vapor Pressure Deficit vs Temperature')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

In [ ]:
# Classify VPD hazard
vpd_example = 38.1
atm_calc = AtmosphericCalculator()
vpd_hazard = atm_calc.classify_vpd_hazard(vpd_example)
print(f"VPD: {vpd_example} hPa")
print(f"Hazard Level: {vpd_hazard['hazard_level']}")

## 4. Terrain Aspect

**Source**: Digital Elevation Model

**Equation**:
$$Asp_{norm} = \frac{1 + \cos(Aspect - 225°)}{2}$$

In [ ]:
# Calculate aspect normalization
terrain_calc = TerrainCalculator()
aspects = np.arange(0, 360, 10)
aspect_norm = [terrain_calc.calculate_aspect_norm(a) for a in aspects]

plt.figure(figsize=(12, 6))
plt.plot(aspects, aspect_norm, 'b-', linewidth=2)
plt.axvline(x=225, color='r', linestyle='--', label='SW aspect (225°) - Maximum hazard')
plt.xlabel('Aspect (degrees)')
plt.ylabel('Normalized Aspect Value')
plt.title('Terrain Aspect Normalization Function')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

In [ ]:
# Get aspect class
aspect_example = 225
aspect_info = terrain_calc.get_aspect_class(aspect_example)
print(f"Aspect: {aspect_example}°")
print(f"Aspect Class: {aspect_info['aspect_class']}")
print(f"ROS Adjustment Factor: {aspect_info['ros_adjustment_factor']}")
print(f"Normalized Value: {aspect_info['normalized_value']:.3f}")

## 5. Drought Code (DC)

**Source**: CFFDRS

**Equation**:
$$DC_t = DC_{t-1} + 0.5 \cdot (T_{max} + 4.0) - P_{eff}$$

In [ ]:
# Calculate Drought Code
dc_calc = DroughtCodeCalculator()

# Simulate DC progression over 30 days
days = np.arange(0, 30)
dc_values = []
dc = 100  # Initial value

for day in days:
    temp = 32 + np.random.randn() * 3  # Random temperature
    precip = max(0, np.random.randn() * 2)  # Random precipitation
    dc = dc_calc.calculate_drought_code(dc, temp, precip)
    dc_values.append(dc)

plt.figure(figsize=(12, 6))
plt.plot(days, dc_values, 'b-', linewidth=2)
plt.axhline(y=400, color='r', linestyle='--', label='Critical threshold (400)')
plt.xlabel('Days')
plt.ylabel('Drought Code')
plt.title('Drought Code Progression')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

In [ ]:
# Classify DC hazard
dc_example = 487
dc_hazard = dc_calc.classify_dc_hazard(dc_example)
print(f"Drought Code: {dc_example}")
print(f"Hazard Level: {dc_hazard['hazard_level']}")

## 6. Canopy Bulk Density (CBD)

**Source**: LiDAR / Forest Inventory

**Equation**:
$$CBD = \frac{\sum B_c}{V_c}$$

In [ ]:
# Classify CBD hazard
fuel_struct = FuelStructureCalculator('pinus_halepensis')

cbd_values = [0.03, 0.08, 0.12, 0.18, 0.25, 0.35]
hazard_levels = []

for cbd in cbd_values:
    hazard = fuel_struct.classify_cbd_hazard(cbd)
    hazard_levels.append(hazard['hazard_level'])
    print(f"CBD = {cbd:.2f} kg/m³ → Hazard Level: {hazard['hazard_level']}")

In [ ]:
print("\n✅ Parameter extraction completed successfully")